In [12]:
def init_model_hidden1():
    global pm_output, pm_hidden, input_cnt, output_cnt, hidden_cnt
    
    pm_hidden = alloc_param_pair([input_cnt, hidden_cnt])
    pm_output = alloc_param_pair([hidden_cnt, output_cnt])
    
def alloc_param_pair(shape):
    weight = np.random.normal(RND_MEAN, RND_STD, shape)
    bias = np.zeros(shape[-1])
    return {'w':weight, 'b':bias}

def forward_neuralnet_hidden1(x):
    global pm_output, pm_hidden
    
    hidden = relu(np.matmul(x, pm_hidden['w']) + pm_hidden['b'])
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
    
    return output, [x, hidden]

def relu(x):
    return np.maximum(x, 0)

def backprop_neuralnet_hidden1(G_output, aux):
    global pm_output, pm_hidden
    
    x, hidden = aux

    g_output_w_out = hidden.transpose()                      
    G_w_out = np.matmul(g_output_w_out, G_output)            
    G_b_out = np.sum(G_output, axis=0)                       

    g_output_hidden = pm_output['w'].transpose()             
    G_hidden = np.matmul(G_output, g_output_hidden)          

    pm_output['w'] -= LEARNING_RATE * G_w_out                
    pm_output['b'] -= LEARNING_RATE * G_b_out                
    
    G_hidden = G_hidden * relu_derv(hidden)
    
    g_hidden_w_hid = x.transpose()                           
    G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)            
    G_b_hid = np.sum(G_hidden, axis=0)                       
    
    pm_hidden['w'] -= LEARNING_RATE * G_w_hid                
    pm_hidden['b'] -= LEARNING_RATE * G_b_hid                
    
def relu_derv(y):
    return np.sign(y)

def init_model_hiddens():
    global pm_output, pm_hiddens, input_cnt, output_cnt, hidden_config
    
    pm_hiddens = []
    prev_cnt = input_cnt
    
    for hidden_cnt in hidden_config:
        pm_hiddens.append(alloc_param_pair([prev_cnt, hidden_cnt]))
        prev_cnt = hidden_cnt
    
    pm_output = alloc_param_pair([prev_cnt, output_cnt])
    
def forward_neuralnet_hiddens(x):
    global pm_output, pm_hiddens
    
    hidden = x
    hiddens = [x]
    
    for pm_hidden in pm_hiddens:
        hidden = relu(np.matmul(hidden, pm_hidden['w']) + pm_hidden['b'])
        hiddens.append(hidden)
        
    output = np.matmul(hidden, pm_output['w']) + pm_output['b']
    
    return output, hiddens

def backprop_neuralnet_hiddens(G_output, aux):
    global pm_output, pm_hiddens

    hiddens = aux
    
    g_output_w_out = hiddens[-1].transpose()
    G_w_out = np.matmul(g_output_w_out, G_output)
    G_b_out = np.sum(G_output, axis=0)

    g_output_hidden = pm_output['w'].transpose() 
    G_hidden = np.matmul(G_output, g_output_hidden)

    pm_output['w'] -= LEARNING_RATE * G_w_out
    pm_output['b'] -= LEARNING_RATE * G_b_out
    
    for n in reversed(range(len(pm_hiddens))):
        G_hidden = G_hidden * relu_derv(hiddens[n+1])

        g_hidden_w_hid = hiddens[n].transpose()
        G_w_hid = np.matmul(g_hidden_w_hid, G_hidden)
        G_b_hid = np.sum(G_hidden, axis=0)
    
        g_hidden_hidden = pm_hiddens[n]['w'].transpose()
        G_hidden = np.matmul(G_hidden, g_hidden_hidden)

        pm_hiddens[n]['w'] -= LEARNING_RATE * G_w_hid
        pm_hiddens[n]['b'] -= LEARNING_RATE * G_b_hid

global hidden_config

def init_model():
    if hidden_config is not None:
        print('은닉 계층 {}개를 갖는 다층 퍼셉트론이 작동되었습니다.'. \
              format(len(hidden_config)))
        init_model_hiddens()
    else:
        print('은닉 계층 하나를 갖는 다층 퍼셉트론이 작동되었습니다.')
        init_model_hidden1()
    
def forward_neuralnet(x):
    if hidden_config is not None:
        return forward_neuralnet_hiddens(x)
    else:
        return forward_neuralnet_hidden1(x)
    
def backprop_neuralnet(G_output, hiddens):
    if hidden_config is not None:
        backprop_neuralnet_hiddens(G_output, hiddens)
    else:
        backprop_neuralnet_hidden1(G_output, hiddens)
        
def set_hidden(info):
    global hidden_cnt, hidden_config
    if isinstance(info, int):
        hidden_cnt = info
        hidden_config = None
    else:
        hidden_config = info

In [13]:
%run Chapter_1.ipynb
set_hidden([])
abalone_exec()

Epoch 1: loss=33.875, accuracy=0.557/0.812
Epoch 2: loss=8.226, accuracy=0.820/0.814
Epoch 3: loss=7.582, accuracy=0.812/0.809
Epoch 4: loss=7.475, accuracy=0.808/0.811
Epoch 5: loss=7.395, accuracy=0.810/0.809
Epoch 6: loss=7.328, accuracy=0.808/0.810
Epoch 7: loss=7.269, accuracy=0.808/0.811
Epoch 8: loss=7.217, accuracy=0.808/0.812
Epoch 9: loss=7.175, accuracy=0.810/0.810
Epoch 10: loss=7.135, accuracy=0.809/0.810

Final Test: final accuracy = 0.810
[[1.02697603]
 [1.47450981]
 [1.66960135]
 [2.04468668]
 [1.62513525]
 [0.60292627]
 [2.39993815]
 [0.54107313]
 [0.46878034]
 [1.01969382]]
[4.16894769]
Epoch 20: loss=5.804, accuracy=0.825/0.831
Epoch 40: loss=5.259, accuracy=0.834/0.828
Epoch 60: loss=5.056, accuracy=0.837/0.838
Epoch 80: loss=4.950, accuracy=0.838/0.840
Epoch 100: loss=4.910, accuracy=0.840/0.826

Final Test: final accuracy = 0.826
Epoch 1: loss=7.436, accuracy=0.801/0.843
Epoch 2: loss=6.422, accuracy=0.815/0.819
Epoch 3: loss=5.951, accuracy=0.822/0.843
Epoch 4: l

In [14]:
set_hidden(4)
abalone_exec(epoch_count=50, report=10)

Epoch 10: loss=5.196, accuracy=0.833/0.844
Epoch 20: loss=5.116, accuracy=0.835/0.831
Epoch 30: loss=5.064, accuracy=0.836/0.801
Epoch 40: loss=5.078, accuracy=0.834/0.843
Epoch 50: loss=5.091, accuracy=0.836/0.822

Final Test: final accuracy = 0.822


In [15]:
abalone_exec(epoch_count=50, report=10)

Epoch 10: loss=5.164, accuracy=0.833/0.835
Epoch 20: loss=5.149, accuracy=0.834/0.842
Epoch 30: loss=5.145, accuracy=0.833/0.853
Epoch 40: loss=5.148, accuracy=0.834/0.827
Epoch 50: loss=5.100, accuracy=0.834/0.848

Final Test: final accuracy = 0.848


In [16]:
%run Chapter_2.ipynb
set_hidden(6)
pulsar_exec(epoch_count=50, report=10)

Epoch 1: loss=33.875, accuracy=0.557/0.812
Epoch 2: loss=8.226, accuracy=0.820/0.814
Epoch 3: loss=7.582, accuracy=0.812/0.809
Epoch 4: loss=7.475, accuracy=0.808/0.811
Epoch 5: loss=7.395, accuracy=0.810/0.809
Epoch 6: loss=7.328, accuracy=0.808/0.810
Epoch 7: loss=7.269, accuracy=0.808/0.811
Epoch 8: loss=7.217, accuracy=0.808/0.812
Epoch 9: loss=7.175, accuracy=0.810/0.810
Epoch 10: loss=7.135, accuracy=0.809/0.810

Final Test: final accuracy = 0.810
[[1.02697603]
 [1.47450981]
 [1.66960135]
 [2.04468668]
 [1.62513525]
 [0.60292627]
 [2.39993815]
 [0.54107313]
 [0.46878034]
 [1.01969382]]
[4.16894769]
Epoch 20: loss=5.804, accuracy=0.825/0.831
Epoch 40: loss=5.259, accuracy=0.834/0.828
Epoch 60: loss=5.056, accuracy=0.837/0.838
Epoch 80: loss=4.950, accuracy=0.838/0.840
Epoch 100: loss=4.910, accuracy=0.840/0.826

Final Test: final accuracy = 0.826
Epoch 1: loss=14.104, result=0.932,0.591,0.869,0.704
Epoch 2: loss=12.449, result=0.973,0.958,0.746,0.839
Epoch 3: loss=11.660, result=0

In [17]:
set_hidden([6])
pulsar_exec(epoch_count=50, report=10)

Epoch 10: loss=10.671, result=0.970,0.972,0.711,0.821
Epoch 20: loss=11.569, result=0.974,0.893,0.827,0.859
Epoch 30: loss=11.590, result=0.977,0.896,0.857,0.876
Epoch 40: loss=11.502, result=0.971,0.968,0.716,0.824
Epoch 50: loss=12.662, result=0.972,0.962,0.740,0.836

Final Test: final result = 0.972,0.962,0.740,0.836


In [18]:
set_hidden([12,6])
pulsar_exec(epoch_count=200, report=40)

Epoch 40: loss=11.605, result=0.977,0.897,0.824,0.859
Epoch 80: loss=11.912, result=0.966,0.969,0.621,0.757
Epoch 120: loss=10.889, result=0.978,0.945,0.788,0.859
Epoch 160: loss=11.983, result=0.978,0.928,0.804,0.862
Epoch 200: loss=10.180, result=0.968,0.970,0.641,0.772

Final Test: final result = 0.968,0.970,0.641,0.772


In [22]:
%run Chapter_3.ipynb

Epoch 1: loss=33.875, accuracy=0.557/0.812
Epoch 2: loss=8.226, accuracy=0.820/0.814
Epoch 3: loss=7.582, accuracy=0.812/0.809
Epoch 4: loss=7.475, accuracy=0.808/0.811
Epoch 5: loss=7.395, accuracy=0.810/0.809
Epoch 6: loss=7.328, accuracy=0.808/0.810
Epoch 7: loss=7.269, accuracy=0.808/0.811
Epoch 8: loss=7.217, accuracy=0.808/0.812
Epoch 9: loss=7.175, accuracy=0.810/0.810
Epoch 10: loss=7.135, accuracy=0.809/0.810

Final Test: final accuracy = 0.810
[[1.02697603]
 [1.47450981]
 [1.66960135]
 [2.04468668]
 [1.62513525]
 [0.60292627]
 [2.39993815]
 [0.54107313]
 [0.46878034]
 [1.01969382]]
[4.16894769]
Epoch 20: loss=5.804, accuracy=0.825/0.831
Epoch 40: loss=5.259, accuracy=0.834/0.828
Epoch 60: loss=5.056, accuracy=0.837/0.838
Epoch 80: loss=4.950, accuracy=0.838/0.840
Epoch 100: loss=4.910, accuracy=0.840/0.826

Final Test: final accuracy = 0.826
Epoch 1: loss=15.866, accuracy=0.311/0.197
Epoch 2: loss=15.464, accuracy=0.328/0.343
Epoch 3: loss=15.464, accuracy=0.328/0.338
Epoch 4

In [23]:
set_hidden(10)
steel_exec(epoch_count=50, report=10)

Epoch 10: loss=15.256, accuracy=0.337/0.345
Epoch 20: loss=15.479, accuracy=0.328/0.210
Epoch 30: loss=15.227, accuracy=0.339/0.343
Epoch 40: loss=15.197, accuracy=0.340/0.430
Epoch 50: loss=15.405, accuracy=0.331/0.179

Final Test: final accuracy = 0.179


In [24]:
set_hidden([12, 6, 4])
steel_exec(epoch_count=50, report=10)

Epoch 10: loss=15.866, accuracy=0.311/0.294
Epoch 20: loss=15.702, accuracy=0.318/0.238
Epoch 30: loss=15.331, accuracy=0.334/0.381
Epoch 40: loss=16.029, accuracy=0.304/0.437
Epoch 50: loss=15.271, accuracy=0.337/0.210

Final Test: final accuracy = 0.210


In [25]:
LEARNING_RATE=0.0001
hidden_config = [12, 6, 4]
steel_exec(epoch_count=50, report=10)

Epoch 10: loss=15.880, accuracy=0.310/0.402
Epoch 20: loss=15.821, accuracy=0.313/0.246
Epoch 30: loss=15.509, accuracy=0.326/0.212
Epoch 40: loss=15.821, accuracy=0.313/0.419
Epoch 50: loss=15.464, accuracy=0.328/0.381

Final Test: final accuracy = 0.381
